In [ ]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime
import copy

In [ ]:
def format_date_time(arr, index) -> None:
    for i in range(len(arr)):
        try:
            arr[i][index] = datetime.datetime.strptime(
                arr[i][index], "%Y-%m-%dT%H:%M:%S.%fZ"
            ) + datetime.timedelta(hours=2)
        except ValueError:
            continue


def apply_cutoff(arr, index, cutoff) -> np.ndarray:
    apply_func = np.vectorize(lambda x: x > cutoff)
    return arr[apply_func(arr[:, index])]


def filter_users_by_role(arr, role) -> np.ndarray:
    return np.array([x for x in arr if x[2]["role"] == role])


def filter_steps_by_users(steps_arr, users_arr):
    user_ids = [x[1] for x in users_arr]
    return np.array([x for x in steps_arr if x[1] in user_ids])

def filter_threads_by_users(threads_arr, users_arr):
    user_ids = [x[1] for x in users_arr]
    return np.array([x for x in threads_arr if x[4] in user_ids])

def filter_steps_by_id(steps_arr, id):
    return np.array([x for x in steps_arr if x[1] != id])

def get_all_steps_by_thread_id(steps_arr, thread_id):
    return np.array([x for x in steps_arr if x[3] == thread_id])

def filter_starters(questions: np.ndarray) -> np.ndarray:
    questions = np.array(
        [
            x
            for x in questions
            if not x.startswith(
                "Wie nennt man es wenn ein Krankenwagen an dir vorbeifährt und der Ton sich so komisch ändert, Wiii---Wooo mäßig?"
            )
        ]
    )
    questions = np.array(
        [
            x
            for x in questions
            if not x.startswith("Erstell mir drei Prüfungsfragen zum Thema '")
        ]
    )
    questions = np.array(
        [
            x
            for x in questions
            if not x.startswith(
                "Schreib mir eine kurze Zusammenfassung über das Thema '"
            )
        ]
    )
    return questions

In [ ]:
with open("db_entries.pkl", "rb") as f:
    data = pickle.load(f)

CUTOFF = datetime.datetime(2021, 6, 24)

## User Analysis

The Users are a List of Tuples.  
Each Tuple has 4 Elements:
- The User ID Internal UUID
- The Users readable Identifier
- The Metadata with Role and auth provider
- The Creation Time

In [ ]:
users = copy.deepcopy(np.array(data["users"]))

format_date_time(users, 3)
# Remove all users created before the day it went live
# These are my accounts used in testing
users = apply_cutoff(users, 3, CUTOFF)
filtered_users = filter_users_by_role(users, "l2pstudent")

print(f"Number of Students: {len(filtered_users)}")

fig = plt.figure(dpi=100, figsize=(10, 5))
sns.histplot(
    filtered_users[:, 3],
    kde=True,
    shrink=0.8,
    edgecolor="none",
    kde_kws={"bw_adjust": 0.35},
)
plt.xlabel("Creation time")
plt.ylabel("Number of users")
plt.title("User creation times")
plt.xticks(rotation=45)
plt.show()

## Step Analysis

A Step is any Action that happens in the System.
This can be a User asking the System, the System answering, The System calling a tool, etc...  
The steplist is a List of Tuples with each tuple having 20 Elements:

- The Step UUID
- The Step Name, e.g. "Landau", "User", "DB Call", "Vector Call", etc...
- The Step Type, e.g. "user_message", "tool", "LLM", etc...
- The Thread ID
- The Parent ID
- The Disable Feedback Flag
- The Streaming Flag
- The Wait for Answer Flag
- The Is Error Flag
- The Metadata
- The Tags
- The Input
- The Output
- The Creation Time
- The Start Time
- The End Time
- The Generation
- If the Input should be shown
- The Language
- The Indent

In [ ]:
steps = copy.deepcopy(np.array(data["steps"]))
for i in [13, 14, 15]:
    format_date_time(steps, i)
    steps = apply_cutoff(steps, i, CUTOFF)

filtered_steps = filter_steps_by_users(steps, filtered_users)

print(f"Number of Questions: {len(filtered_steps)}")

user_names, counts = np.unique([step[1] for step in filtered_steps], return_counts=True)
# sorted used names
user_names = user_names[np.argsort(counts)[::-1]]
counts = counts[np.argsort(counts)[::-1]]
x = np.arange(len(counts))

fig = plt.figure(dpi=100, figsize=(10, 5))
plt.bar(x, counts)
plt.xlabel("User")
plt.ylabel("Number of Questions")
plt.title("Number of questions asked by each student, total: " + str(len(filtered_steps)))
plt.xlim(-1, len(counts))
plt.show()

In [ ]:
steps = copy.deepcopy(np.array(data["steps"]))
for i in [13, 14, 15]:
    format_date_time(steps, i)
    steps = apply_cutoff(steps, i, CUTOFF)

filtered_steps = filter_steps_by_users(steps, filtered_users)

fig = plt.figure(dpi=100, figsize=(10, 5))
sns.histplot(
    data=filtered_steps[:, 13],
    bins=25,
    kde=True,
    kde_kws={"bw_adjust": 0.35},
    shrink=0.8,
    edgecolor="none",
)
plt.xlabel("Time of Question")
plt.ylabel("Number of questions")
plt.title("When questions were asked")
plt.xticks(rotation=45)
plt.show()

In [ ]:
user_questions = np.array(filtered_steps[:, 12])
user_questions = filter_starters(user_questions)
print(f"Number of Questions: {len(user_questions)}")

for question in user_questions:
    print(question)
    print("-" * 10)

In [ ]:
tools = [
    "Landau",
    "Inhaltsverzeichnis",
    "Kapitel Abfrage",
    "Formel Abfrage",
    "Vektorsuche",
]

tool_uses = {tool: 0 for tool in tools}
for step in steps:
    for tool in tools:
        if tool in step[1]:
            if tool == "Landau LLM":
                tool_uses["Landau"] += 1
                continue
            tool_uses[tool] += 1

# sort the dictionary by value
tool_uses = dict(sorted(tool_uses.items(), key=lambda item: item[1], reverse=True))

fig = plt.figure(dpi=100, figsize=(10, 5))
plt.bar(tool_uses.keys(), tool_uses.values())
plt.xlabel("Tool")
plt.ylabel("Number of uses")
plt.title("Number of uses of each tool")
plt.show()

## Feedbacks

Feedbacks are a List of Tuples with each Tuple having 5 Elements:

- The Feedback UUID
- The ID of the Step the Feedback is for
- The ID of the Thread the Feedback is for
- The Value of the Feedback, either 1 for positive or 0 for negative
- The Comment of the Feedback

In [ ]:
feedbacks = copy.deepcopy(np.array(data["feedbacks"]))

positive_feedbacks = [fb for fb in feedbacks if fb[3] == 1]
negative_feedbacks = [fb for fb in feedbacks if fb[3] == 0]

print(f"Number of positive feedbacks: {len(positive_feedbacks)}")
print(f"Number of negative feedbacks: {len(negative_feedbacks)}")
print()

print("# Positive feedbacks:")
for fb in positive_feedbacks:
    # for step in steps:
    #     if step[0] == fb[1]:
    #         print(f"Step: {step[12]}")
    if fb[4]:
        print(f"- {fb[4]}")
print()

print("# Negative feedbacks:")
for fb in negative_feedbacks:
    # for step in steps:
    #     if step[0] == fb[1]:
    #         print(f"Step: {step[12]}")
    if fb[4]:
        print(f"- {fb[4]}")

# Generate feedback markdown

output_string = "# Feedback\n\n"
output_string += "## Positive feedbacks\n\n"
for i, fb in enumerate(positive_feedbacks):
    step_id = fb[1]
    comment = fb[4]
    LLM_output = None
    for step in steps:
        if step[0] == step_id:
            LLM_output = step[12]

    output_string += f"### Positives Feedback {i+1}\n\n"
    if comment:
        output_string += f"> Comment:\n{comment}\n\n"
    if LLM_output:
        output_string += f"> LLM Output: {LLM_output}\n\n"

output_string += "## Negative feedbacks\n\n"
for i, fb in enumerate(negative_feedbacks):
    step_id = fb[1]
    comment = fb[4]
    LLM_output = None
    for step in steps:
        if step[0] == step_id:
            LLM_output = step[12]

    output_string += f"### Negatives Feedback {i+1}\n\n"
    if comment:
        output_string += f"> Comment:\n{comment}\n\n"
    if LLM_output:
        output_string += f"LLM Output:\n{LLM_output}\n\n"

with open("feedback.md", "w") as f:
    f.write(output_string)

## Threads

"id" UUID PRIMARY KEY,  
"createdAt" TEXT,  
"name" TEXT,  
"userId" UUID,  
"userIdentifier" TEXT,  
"tags" TEXT[],   
"metadata" JSONB,  

In [ ]:
threads = copy.deepcopy(np.array(data["threads"]))
filtered_threads = filter_threads_by_users(threads, filtered_users)

print(f"Number of threads: {len(filtered_threads)}")

thread_lengths = [len(get_all_steps_by_thread_id(steps, thread[0])) for thread in filtered_threads]

plt.hist(thread_lengths, bins=25)
print(f"Average thread length: {np.mean(thread_lengths)}")
print(f"Median thread length: {np.median(thread_lengths)}")
print(f"Max thread length: {np.max(thread_lengths)}")
print(f"Min thread length: {np.min(thread_lengths)}")
plt.show()

In [ ]:
output = ""
for thread in filtered_threads:
    thread_steps = get_all_steps_by_thread_id(steps, thread[0])
    if len(thread_steps) < 2:
        continue
    thread_steps = thread_steps[np.argsort(thread_steps[:, 13])]
    # sort the steps by time
    if len(thread_steps) == 227:
        for seq in thread_steps:
            output += f"{seq[12]}\n"
            output += "-------------------------------------------------------------\n"
        
with open("longest_thread.txt", "w") as f:
    f.write(output)